# NCAR Innovator Program Project

### Notebook by: Maria J. Molina (NCAR/University of Maryland) and Jessica Moulite (Howard University)

Import packages

In [1]:
import datetime
import numpy as np
import pandas as pd
import xarray as xr

In [2]:
def cel_to_far(tmp):
    """
    Convert celsius to farenheit
    """
    newtmp = []
    for i in tmp:
        if i == 99.9:
            newtmp.append(np.nan)
        if i != 99.9:
            newtmp.append((i * (9/5)) + 32)
    return np.array(newtmp)


def cel_to_nan(tmp):
    """
    Change 9999 to np.nan
    """
    newtmp = []
    for i in tmp:
        if i == 99.9:
            newtmp.append(np.nan)
        if i != 99.9:
            newtmp.append(i)
    return np.array(newtmp)


def convert_to_datetime(time_value):
    """
    Convert to a datetime object to be able to compute time difference.
    """
    return datetime.datetime(
        pd.to_datetime(time_value).year,
        pd.to_datetime(time_value).month,
        pd.to_datetime(time_value).day,
        pd.to_datetime(time_value).hour,
        pd.to_datetime(time_value).minute,
        pd.to_datetime(time_value).second,
    )


def compute_time_difference(a, b):
    """
    Compute time difference between two times.
    """
    if a >= b:
        return convert_to_datetime(a) - convert_to_datetime(b)
    if b > a:
        return convert_to_datetime(b) - convert_to_datetime(a)
    
    
def extract_era5_time(t_daily, t_full):
    """
    Extract the era5 temperature corresponding time.
    """
    era5_time_array = []
    
    for i, j in zip(t_daily.time, t_daily):
        
        tmp = t_full.sel(
            time=slice(i, pd.to_datetime(i.values) + datetime.timedelta(hours=23)))
        
        era5_time_array.append(tmp.where(tmp == j, drop=True).time.values)
        
    return era5_time_array

### load in our csv file with temperatures from Shading Dade

In [3]:
Shading_dade_temp = pd.read_csv('/Users/molina/Desktop/ShadingDade_NOAA_KMIA_Temperature.csv')

In [4]:
Shading_dade_temp.columns

Index(['Datetime', 'ShadingDade_Temperature_Daily_Max (C)',
       'ShadingDade_Temperature_Daily_Min (C)',
       'NOAA_KMIA_Temperature_Daily_Max (C)',
       'NOAA_KMIA_Temperature_Daily_Min (C)',
       'ShadingDade_Temperature_Daily_Max (F)',
       'ShadingDade_Temperature_Daily_Min (F)',
       'NOAA_KMIA_Temperature_Daily_Max (F)',
       'NOAA_KMIA_Temperature_Daily_Min (F)',
       'Daily_Temperature_Max_Difference (C)',
       'Daily_Temperature_Min_Difference (C)',
       'Daily_Temperature_Max_Difference (F)',
       'Daily_Temperature_Min_Difference (F)',
       'Corresponding_Datetime_ShadingDade_DailyMax',
       'Corresponding_Datetime_ShadingDade_DailyMin',
       'Corresponding_Datetime_NOAA_KMIA_DailyMax',
       'Corresponding_Datetime_NOAA_KMIA_DailyMin',
       'Corresponding_Datetime_NOAA_KMIA_DailyMax_Difference (C)',
       'Corresponding_Datetime_NOAA_KMIA_DailyMax_Difference (F)',
       'Corresponding_Datetime_NOAA_KMIA_DailyMin_Difference (C)',
       'Cor

In [5]:
# location of little haiti temperature sensor
SD_latitude  = 25.8436392
SD_longitude = -80.1816499 % 360

In [6]:
alltmaxtimes = pd.to_datetime(Shading_dade_temp['Corresponding_Datetime_ShadingDade_DailyMax'].values)

In [7]:
jun_tmaxtimes = alltmaxtimes[alltmaxtimes.month == 6]
jul_tmaxtimes = alltmaxtimes[alltmaxtimes.month == 7]
aug_tmaxtimes = alltmaxtimes[alltmaxtimes.month == 8]
sep_tmaxtimes = alltmaxtimes[alltmaxtimes.month == 9]

In [8]:
alltmintimes = pd.to_datetime(Shading_dade_temp['Corresponding_Datetime_ShadingDade_DailyMin'].values)

In [9]:
jun_tmintimes = alltmintimes[alltmintimes.month == 6]
jul_tmintimes = alltmintimes[alltmintimes.month == 7]
aug_tmintimes = alltmintimes[alltmintimes.month == 8]
sep_tmintimes = alltmintimes[alltmintimes.month == 9]

### ERA5 work

Download source for ERA5: https://rda.ucar.edu/datasets/ds633.0/

ERA5 (ECMWF -- European Centre for Medium-Range Weather Forecasts -- fifth reanalysis) reference: https://rmets.onlinelibrary.wiley.com/doi/full/10.1002/qj.3803

In [10]:
# open files
ds_jun = xr.open_dataset(
    '/Users/molina/Desktop/e5.oper.an.sfc.128_167_2t.ll025sc.2019060100_2019063023.nc')['VAR_2T']
ds_jul = xr.open_dataset(
    '/Users/molina/Desktop/e5.oper.an.sfc.128_167_2t.ll025sc.2019070100_2019073123.nc')['VAR_2T']
ds_aug = xr.open_dataset(
    '/Users/molina/Desktop/e5.oper.an.sfc.128_167_2t.ll025sc.2019080100_2019083123.nc')['VAR_2T']
ds_sep = xr.open_dataset(
    '/Users/molina/Desktop/e5.oper.an.sfc.128_167_2t.ll025sc.2019090100_2019093023.nc')['VAR_2T']

### ERA5 temperature during the Shading Dade daily maximum (2019)

In [11]:
era5_temp_corresponding_sdmax_jun = ds_jun.sel(
    longitude=SD_longitude, latitude=SD_latitude, time = jun_tmaxtimes, method='nearest') - 273.15
era5_temp_corresponding_sdmax_jul = ds_jul.sel(
    longitude=SD_longitude, latitude=SD_latitude, time = jul_tmaxtimes, method='nearest') - 273.15
era5_temp_corresponding_sdmax_aug = ds_aug.sel(
    longitude=SD_longitude, latitude=SD_latitude, time = aug_tmaxtimes, method='nearest') - 273.15
era5_temp_corresponding_sdmax_sep = ds_sep.sel(
    longitude=SD_longitude, latitude=SD_latitude, time = sep_tmaxtimes, method='nearest') - 273.15

In [12]:
era5_temp_corresponding_sdmax = xr.concat([
    era5_temp_corresponding_sdmax_jun,
    era5_temp_corresponding_sdmax_jul,
    era5_temp_corresponding_sdmax_aug,
    era5_temp_corresponding_sdmax_sep,
], dim='time')

### differences (temperature)

In [13]:
# difference between shading dade temperature maximum and the era5 corresponding temperature at that time (C)
tmax_daily_difference_C = Shading_dade_temp[
    'ShadingDade_Temperature_Daily_Max (C)'] - era5_temp_corresponding_sdmax.values

In [15]:
# difference between shading dade temperature maximum and the era5 corresponding temperature at that time (F)
tmax_daily_difference_F = Shading_dade_temp['ShadingDade_Temperature_Daily_Max (F)'] - cel_to_far(
    era5_temp_corresponding_sdmax.values)

### ERA5 temperature during the Shading Dade daily minimum (2019)

In [19]:
era5_temp_corresponding_sdmin_jun = ds_jun.sel(
    longitude=SD_longitude, latitude=SD_latitude, time = jun_tmintimes, method='nearest') - 273.15
era5_temp_corresponding_sdmin_jul = ds_jul.sel(
    longitude=SD_longitude, latitude=SD_latitude, time = jul_tmintimes, method='nearest') - 273.15
era5_temp_corresponding_sdmin_aug = ds_aug.sel(
    longitude=SD_longitude, latitude=SD_latitude, time = aug_tmintimes, method='nearest') - 273.15
era5_temp_corresponding_sdmin_sep = ds_sep.sel(
    longitude=SD_longitude, latitude=SD_latitude, time = sep_tmintimes, method='nearest') - 273.15

In [20]:
era5_temp_corresponding_sdmin = xr.concat([
    era5_temp_corresponding_sdmin_jun,
    era5_temp_corresponding_sdmin_jul,
    era5_temp_corresponding_sdmin_aug,
    era5_temp_corresponding_sdmin_sep,
], dim='time')

In [21]:
# difference between shading dade temperature minimum and the era5 corresponding temperature at that time (C)
tmin_daily_difference_C = Shading_dade_temp[
    'ShadingDade_Temperature_Daily_Min (C)'] - era5_temp_corresponding_sdmin.values

In [22]:
# difference between shading dade temperature minimum and the era5 corresponding temperature at that time (F)
tmin_daily_difference_F = Shading_dade_temp['ShadingDade_Temperature_Daily_Min (F)'] - cel_to_far(
    era5_temp_corresponding_sdmin.values)

### ERA5 daily temperature maximum (2019)

In [25]:
era5_temp_jun = ds_jun.sel(
    longitude=SD_longitude, latitude=SD_latitude, method='nearest') - 273.15
era5_temp_jul = ds_jul.sel(
    longitude=SD_longitude, latitude=SD_latitude, method='nearest') - 273.15
era5_temp_aug = ds_aug.sel(
    longitude=SD_longitude, latitude=SD_latitude, method='nearest') - 273.15
era5_temp_sep = ds_sep.sel(
    longitude=SD_longitude, latitude=SD_latitude, method='nearest') - 273.15

In [26]:
era5_tmax_jun = era5_temp_jun.resample(time='D').max()
era5_tmax_jul = era5_temp_jul.resample(time='D').max()
era5_tmax_aug = era5_temp_aug.resample(time='D').max()
era5_tmax_sep = era5_temp_sep.resample(time='D').max()

In [27]:
era5_tmax = xr.concat([
    era5_tmax_jun,
    era5_tmax_jul,
    era5_tmax_aug,
    era5_tmax_sep,
], dim='time')

In [28]:
era5_tmax = era5_tmax.sel(time=slice(alltmaxtimes.date[0], alltmaxtimes.date[-1]))

### ERA5 daily temperature minimum (2019)

In [30]:
era5_tmin_jun = era5_temp_jun.resample(time='D').min()
era5_tmin_jul = era5_temp_jul.resample(time='D').min()
era5_tmin_aug = era5_temp_aug.resample(time='D').min()
era5_tmin_sep = era5_temp_sep.resample(time='D').min()

In [31]:
era5_tmin = xr.concat([
    era5_tmin_jun,
    era5_tmin_jul,
    era5_tmin_aug,
    era5_tmin_sep,
], dim='time')

In [32]:
era5_tmin = era5_tmin.sel(time=slice(alltmintimes.date[0], alltmintimes.date[-1]))

### ERA5 daily temperature maximum time (2019)

In [33]:
era5_tmax_jun_time = extract_era5_time(era5_tmax_jun, era5_temp_jun)
era5_tmax_jul_time = extract_era5_time(era5_tmax_jul, era5_temp_jul)
era5_tmax_aug_time = extract_era5_time(era5_tmax_aug, era5_temp_aug)
era5_tmax_sep_time = extract_era5_time(era5_tmax_sep, era5_temp_sep)

In [34]:
era5_tmax_all_time = pd.to_datetime(
    np.vstack([era5_tmax_jun_time,
               era5_tmax_jul_time,
               era5_tmax_aug_time,
               era5_tmax_sep_time])[:,0])

In [35]:
era5_tmax_time = era5_tmax_all_time[(
    era5_tmax_all_time>='2019-06-08') & (era5_tmax_all_time<'2019-09-02')]

In [39]:
# absolute time difference between daily maximum temperature (ERA5 and Shading Dade)
tmax_diff = []
for i, j in zip(
    pd.to_datetime(Shading_dade_temp['Corresponding_Datetime_ShadingDade_DailyMax']), era5_tmax_time):
    tmax_diff.append(compute_time_difference(i, j))
tmax_diff = np.array(tmax_diff)

### ERA5 daily temperature minimum time (2019)

In [36]:
era5_tmin_jun_time = extract_era5_time(era5_tmin_jun, era5_temp_jun)
era5_tmin_jul_time = extract_era5_time(era5_tmin_jul, era5_temp_jul)
era5_tmin_aug_time = extract_era5_time(era5_tmin_aug, era5_temp_aug)
era5_tmin_sep_time = extract_era5_time(era5_tmin_sep, era5_temp_sep)

In [37]:
era5_tmin_all_time = pd.to_datetime(
    np.vstack([era5_tmin_jun_time,
               era5_tmin_jul_time,
               era5_tmin_aug_time,
               era5_tmin_sep_time])[:,0])

In [38]:
era5_tmin_time = era5_tmin_all_time[(
    era5_tmin_all_time>='2019-06-08') & (era5_tmin_all_time<'2019-09-02')]

In [40]:
# absolute time difference between daily minimum temperature (ERA5 and Shading Dade)
tmin_diff = []
for i, j in zip(
    pd.to_datetime(Shading_dade_temp['Corresponding_Datetime_ShadingDade_DailyMin']), era5_tmin_time):
    tmin_diff.append(compute_time_difference(i, j))
tmin_diff = np.array(tmin_diff)

### create single csv file

In [41]:
data = {
'ShadingDade_Temperature_Daily_Max (C)' : Shading_dade_temp['ShadingDade_Temperature_Daily_Max (C)'],
'ShadingDade_Temperature_Daily_Max (F)' : Shading_dade_temp['ShadingDade_Temperature_Daily_Max (F)'],
    
'ShadingDade_Temperature_Daily_Min (C)' : Shading_dade_temp['ShadingDade_Temperature_Daily_Min (C)'],
'ShadingDade_Temperature_Daily_Min (F)' : Shading_dade_temp['ShadingDade_Temperature_Daily_Min (F)'],
    
'ERA5_Corresponding_Temperature_During_ShadingDade_Daily_Max (C)': era5_temp_corresponding_sdmax.values,
'ERA5_Corresponding_Temperature_During_ShadingDade_Daily_Max (F)': cel_to_far(era5_temp_corresponding_sdmax.values),
    
'ERA5_Corresponding_Temperature_During_ShadingDade_Daily_Min (C)': era5_temp_corresponding_sdmin.values,
'ERA5_Corresponding_Temperature_During_ShadingDade_Daily_Min (F)': cel_to_far(era5_temp_corresponding_sdmin.values),

'Daily_Temperature_Max_Difference (C)': tmax_daily_difference_C.values,
'Daily_Temperature_Max_Difference (F)': tmax_daily_difference_F.values,
    
'Daily_Temperature_Min_Difference (C)': tmin_daily_difference_C.values,
'Daily_Temperature_Min_Difference (F)': tmin_daily_difference_F.values,
    
'ERA5_Temperature_Daily_Max (C)': era5_tmax.values,
'ERA5_Temperature_Daily_Max (F)': cel_to_far(era5_tmax.values),
'ERA5_Temperature_Daily_Max_Datetime': era5_tmax_time,
    
'ERA5_Temperature_Daily_Min (C)': era5_tmin.values,
'ERA5_Temperature_Daily_Min (F)': cel_to_far(era5_tmin.values),
'ERA5_Temperature_Daily_Min_Datetime': era5_tmin_time,
    
'Absolute_time_difference_tmax_ERA5_ShadingDade (day H:M:S)': tmax_diff,
'Absolute_time_difference_tmin_ERA5_ShadingDade (day H:M:S)': tmin_diff,

       }

In [42]:
df_FINAL = pd.DataFrame(data)

In [43]:
df_FINAL

,ShadingDade_Temperature_Daily_Max (C),ShadingDade_Temperature_Daily_Max (F),ShadingDade_Temperature_Daily_Min (C),ShadingDade_Temperature_Daily_Min (F),ERA5_Corresponding_Temperature_During_ShadingDade_Daily_Max (C),ERA5_Corresponding_Temperature_During_ShadingDade_Daily_Max (F),ERA5_Corresponding_Temperature_During_ShadingDade_Daily_Min (C),ERA5_Corresponding_Temperature_During_ShadingDade_Daily_Min (F),Daily_Temperature_Max_Difference (C),Daily_Temperature_Max_Difference (F),Daily_Temperature_Min_Difference (C),Daily_Temperature_Min_Difference (F),ERA5_Temperature_Daily_Max (C),ERA5_Temperature_Daily_Max (F),ERA5_Temperature_Daily_Max_Datetime,ERA5_Temperature_Daily_Min (C),ERA5_Temperature_Daily_Min (F),ERA5_Temperature_Daily_Min_Datetime,Absolute_time_difference_tmax_ERA5_ShadingDade (day H:M:S),Absolute_time_difference_tmin_ERA5_ShadingDade (day H:M:S)
0,34.875,94.775,23.000,73.400,27.618988,81.714178,27.076782,80.738208,7.256012,13.060822,-4.076782,-7.338208,30.230072,86.414130,2019-06-08 16:00:00,25.743805,78.338849,2019-06-08 22:00:00,0 days 04:00:00,0 days 11:00:01
1,33.750,92.750,25.125,77.225,26.792572,80.226630,27.670563,81.807013,6.957428,12.523370,-2.545563,-4.582013,28.871124,83.968024,2019-06-09 16:00:00,26.113495,79.004291,2019-06-09 11:00:00,0 days 04:00:00,0 days 10:00:00
2,38.500,101.300,23.250,73.850,26.699921,80.059857,26.314667,79.366400,11.800079,21.240143,-3.064667,-5.516400,29.251984,84.653571,2019-06-10 19:00:00,25.005402,77.009723,2019-06-10 09:00:00,0 days 03:00:00,0 days 02:00:00
3,40.750,105.350,24.250,75.650,27.057831,80.704095,26.604706,79.888470,13.692169,24.645905,-2.354706,-4.238470,30.506561,86.911810,2019-06-11 18:00:00,25.968292,78.742926,2019-06-11 09:00:00,0 days 06:00:00,0 days 06:00:00
4,40.250,104.450,24.875,76.775,26.922882,80.461188,26.931915,80.477448,13.327118,23.988812,-2.056915,-3.702448,30.096771,86.174188,2019-06-12 18:00:00,26.637909,79.948236,2019-06-12 08:00:00,0 days 07:00:01,0 days 04:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,40.750,105.350,24.625,76.325,28.515808,83.328455,26.947937,80.506287,12.234192,22.021545,-2.322937,-4.181287,31.926544,89.467780,2019-08-28 19:00:00,26.544891,79.780804,2019-08-28 11:00:00,0 days 05:00:01,0 days 05:00:00
82,41.375,106.475,24.750,76.550,27.561676,81.611017,27.436249,81.385248,13.813324,24.863983,-2.686249,-4.835248,29.879425,85.782965,2019-08-29 16:00:00,26.211426,79.180566,2019-08-29 08:00:00,0 days 03:00:00,0 days 04:00:00
83,35.000,95.000,23.875,74.975,27.451202,81.412164,26.376740,79.478131,7.548798,13.587836,-2.501740,-4.503131,29.632477,85.338458,2019-08-30 22:00:00,26.315643,79.368158,2019-08-30 07:00:00,0 days 08:00:01,0 days 00:59:59
84,36.500,97.700,26.750,80.150,28.057770,82.503986,27.519745,81.535541,8.442230,15.196014,-0.769745,-1.385541,29.950867,85.911560,2019-08-31 22:00:00,27.061188,80.710138,2019-08-31 04:00:00,0 days 09:00:00,0 days 03:59:59


### save csv file

In [44]:
df_FINAL.to_csv('/Users/molina/Desktop/ShadingDade_ERA5_Temperature.csv')